# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [167]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [168]:
# 2) seu código aqui
sinasc.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [169]:
# 3) seu código aqui
dfsi = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
dfsi[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isnull().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [170]:
# 4) seu código aqui
#ver quais linhas são missing
#sinasc['APGAR5'][sinasc.APGAR5.isna()]

dfsi_dup = sinasc['APGAR5'].dropna()
quant = dfsi_dup.count()
dup = dfsi_dup.isna().sum()
print(quant, dup)

26925 0


In [171]:
# 5) seu código aqui
dfsi_dup = dfsi[['ESTCIVMAE', 'CONSULTAS']].fillna(9)
dfsi_dup.isna().sum()

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [172]:
# 6) Seu código aqui
dfsi['QTDFILVIVO'].fillna(0)

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [173]:
# 7) seu código aqui
dfsi['LOCNASC'].fillna(1)
dfsi['IDADEMAE'].fillna(dfsi['IDADEMAE'].mean())
dfsi['ESCMAE'].fillna('Ignorado')
dfsi['GESTACAO'].fillna('Ignorado')
dfsi['GRAVIDEZ'].fillna('Ignorado')
dfsi

LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          4     9.0  
2      37 a 41 semanas    Única          4    10.0  
3      37 a 41 semanas    Única          3    10.0  
4      37 a 41 semanas    Única          4    10.0  
...                ...      ...        ...     ...  
27023  32 a 36 semanas    Única          4     9.0  
27024  37 a 41 semanas    Única          4     9.0  
27025  37 a 41 semanas    Única          4    10.0  
27026  32 a 36 semanas    Única          4     9.0  
27027  37 a 41 semanas    Única          4     8.0  

[27028 rows x 9 columns]

In [174]:
# 8) seu código aqui
'''
Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
'''
dfsi.loc[dfsi['APGAR5'] <= 3, 'apgar5_cat'] = 'asfixia_severa'
dfsi.loc[dfsi['APGAR5'] >= 4, 'apgar5_cat'] = 'asfixia_moderada'
dfsi.loc[dfsi['APGAR5'] >= 6, 'apgar5_cat'] = 'asfixia_leve'
dfsi.loc[dfsi['APGAR5'] >= 8, 'apgar5_cat'] = 'normal'
norm = dfsi['apgar5_cat'][dfsi['apgar5_cat'] == 'normal'].count()
leve = dfsi['apgar5_cat'][dfsi['apgar5_cat'] == 'asfixia_leve'].count()
modera = dfsi['apgar5_cat'][dfsi['apgar5_cat'] == 'asfixia_moderada'].count()
severa = dfsi['apgar5_cat'][dfsi['apgar5_cat'] == 'asfixia_severa'].count()
ttl = norm+leve+modera+severa
print(norm/ttl, leve/ttl, modera/ttl, severa/ttl)

0.9828412256267409 0.011884865366759517 0.0025255338904363973 0.002748375116063138


C:\Users\joao\AppData\Local\Temp\ipykernel_25656\3124116833.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsi.loc[dfsi['APGAR5'] <= 3, 'apgar5_cat'] = 'asfixia_severa'


In [175]:
# 9) seu código aqui
dfsi.columns = ['local_nascimento', 'idade_mae', 'est_civil_mae', 'escolaridade_mae', 'quant_filho_vivo', 
 'tmp_gestacao', 'gravidez', 'consultas', 'apgar_5_min', 'apgar5_cat']
dfsi

local_nascimento  idade_mae  est_civil_mae escolaridade_mae  \
0                     1         19            5.0      8 a 11 anos   
1                     1         29            2.0      8 a 11 anos   
2                     1         37            9.0      8 a 11 anos   
3                     1         30            5.0  12 anos ou mais   
4                     1         30            2.0      8 a 11 anos   
...                 ...        ...            ...              ...   
27023                 1         32            2.0  12 anos ou mais   
27024                 1         19            1.0      8 a 11 anos   
27025                 1         24            2.0      8 a 11 anos   
27026                 1         21            2.0      8 a 11 anos   
27027                 1         18            5.0      8 a 11 anos   

       quant_filho_vivo     tmp_gestacao gravidez  consultas  apgar_5_min  \
0                   0.0  37 a 41 semanas    Única          4         10.0   
1                   1.0  37 a 41 semanas    Única          4          9.0   
2                   2.0  37 a 41 semanas    Única          4         10.0   
3                   0.0  37 a 41 semanas    Única          3         10.0   
4                   1.0  37 a 41 semanas    Única          4         10.0   
...                 ...              ...      ...        ...          ...   
27023               1.0  32 a 36 semanas    Única          4          9.0   
27024               0.0  37 a 41 semanas    Única          4          9.0   
27025               0.0  37 a 41 semanas    Única          4         10.0   
27026               1.0  32 a 36 semanas    Única          4          9.0   
27027               1.0  37 a 41 semanas    Única          4          8.0   

      apgar5_cat  
0         normal  
1         normal  
2         normal  
3         normal  
4         normal  
...          ...  
27023     normal  
27024     normal  
27025     normal  
27026     normal  
27027     normal  

[27028 rows x 10 columns]